# Commits background work

I am restarting this whole script in order to build up piece-by-piece for what I want. I am getting lost in the details of editing old code.

In [ ]:
import subprocess
from os import path
from dateutil.parser import parse as parsedate

import pandas as pd

In [ ]:
parameters = pd.read_csv('repositories.csv',
                         index_col=False)
parameters

In [ ]:
repository_path = parameters['Repository path']
repository_path

In [ ]:
df = []
def dataframes(repository_path):
    '''
    Create a dataframe with count column for each repository.
    '''
    history = subprocess.check_output(
        ['git', 'log', '--pretty=%aI', '--author=Gilles'], 
        cwd=path.expanduser(f'~/documents/websites/{repository_path}'),
        universal_newlines=True).splitlines()
    dates = list(map(parsedate, history))
    df = pd.DataFrame.from_dict({'Date': dates},
                                dtype='datetime64[ns]')\
                     .set_index('Date')
    df['count'] = 1
    commits_daily = df.groupby(df.index.date).count()
    # commits_daily = commits_daily[-90:].reset_index()
    commits_daily = commits_daily.reset_index()\
                                 .rename(columns={'index':'date'})
    return commits_daily

In [ ]:
for item in repository_path:
    commits_daily = dataframes(item)
    print(f'\nrepository path: {item}\n{commits_daily.tail()}')

In [ ]:
dfa = pd.DataFrame({'date': ['2019-05-01', '2019-05-02', '2019-05-03',
                             '2019-05-05', '2019-05-07'],
                    'count': [1, 3, 5, 7, 9]})
dfa

In [ ]:
dfb = pd.DataFrame({'date': ['2019-04-30', '2019-05-01', '2019-05-04',
                             '2019-05-06', '2019-05-07'],
                    'count': [2, 4, 6, 8, 10]})
dfb

In [ ]:
dfc = pd.DataFrame({'date': ['2019-04-29', '2019-05-07'],
                    'count': [3, 4]})
dfc

In [ ]:
df = pd.merge(dfa, dfb, how='outer', on='date')\
       .sort_values(by=['date'])
df

In [ ]:
df['count'] = df['count_x'].fillna(0) + df['count_y'].fillna(0)
df = df.drop(columns=['count_x', 'count_y'])
df

In [ ]:
df = pd.merge(df, dfc, how='outer', on='date')\
       .sort_values(by=['date'])
df

In [ ]:
df['count'] = df['count_x'].fillna(0) + df['count_y'].fillna(0)
df = df.drop(columns=['count_x', 'count_y'])
df